In [1]:
import polars as pl
import numpy as np
import plotly_express as px

marketing = pl.read_csv('marketing.csv')
print(marketing.describe())   

marketing = marketing.with_columns(pl.col(["date_served", "date_subscribed","date_canceled"]).str.to_date("%m/%d/%Y"))
print(marketing.head())

shape: (9, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ user_id   ┆ date_serv ┆ marketing ┆ … ┆ date_subs ┆ date_canc ┆ subscribi ┆ is_retai │
│ ---       ┆ ---       ┆ ed        ┆ _channel  ┆   ┆ cribed    ┆ eled      ┆ ng_channe ┆ ned      │
│ str       ┆ str       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ l         ┆ ---      │
│           ┆           ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ ---       ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ str       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 10037     ┆ 10021     ┆ 10022     ┆ … ┆ 1856      ┆ 577       ┆ 1856      ┆ 1856.0   │
│ null_coun ┆ 0         ┆ 16        ┆ 15        ┆ … ┆ 8181      ┆ 9460      ┆ 8181      ┆ 8181.0   │
│ t         ┆           ┆           ┆           ┆   ┆           ┆           

In [2]:
daily_users = marketing[['date_served','user_id']].sort('date_served').group_by(['date_served']).agg(pl.col('user_id').n_unique().alias("users_num"))
print(daily_users.head())

fig = px.line(
    daily_users,
    x='date_served',
    y='users_num',
    title='Daily Users',
    template='plotly_white',
    labels={'date_served': 'Date Served', 'users_num': 'Number of Users'},
    markers=True,
    text='users_num'
)
fig.update_traces({'line_color':'purple','textposition':'top center'})
fig.update_layout(yaxis=dict(range=[0, None]))
fig.show()

shape: (5, 2)
┌─────────────┬───────────┐
│ date_served ┆ users_num │
│ ---         ┆ ---       │
│ date        ┆ u32       │
╞═════════════╪═══════════╡
│ null        ┆ 16        │
│ 0018-01-01  ┆ 362       │
│ 0018-01-02  ┆ 374       │
│ 0018-01-03  ┆ 348       │
│ 0018-01-04  ┆ 323       │
└─────────────┴───────────┘


In [3]:
total = marketing['user_id'].n_unique()
subscribers = marketing.filter(pl.col('converted')==True)['user_id'].n_unique()
conversion_rate = subscribers/total
print("Conversion rate", round(conversion_rate*100, 2), "%",sep=" ")

retained = marketing.filter(pl.col('is_retained')==True)['user_id'].n_unique()
retention_rate = retained/subscribers
print("Retention rate", round(retention_rate*100, 2), "%",sep=" ")

Conversion rate 13.89 %
Retention rate 66.8 %


In [4]:
def conversion_rate(dataframe, column_names):
    column_conv = dataframe.filter(pl.col('converted')==True).group_by(column_names).agg(pl.col('user_id').n_unique().alias("users_converted"))
    column_total = dataframe.group_by(column_names).agg(pl.col('user_id').n_unique().alias("users_total"))
 
    conversion_df = column_conv.join(column_total, on=column_names, how='inner')
    conversion_df = conversion_df.with_columns(((pl.col("users_converted")/pl.col("users_total")).fill_nan(0)).alias("conversion_rate"))
    return conversion_df

In [5]:
language_conversion_rate = conversion_rate(marketing,'language_displayed')
print('Speaker conversion rate by language: ',language_conversion_rate, sep="\n")

fig = px.bar(
    language_conversion_rate.sort("conversion_rate"), 
    x='language_displayed', 
    y='conversion_rate',
    color_discrete_sequence=['green'],  
    title='Conversion Rate by Language',
    template='plotly_white',
    labels={'language_displayed': 'Language Displayed', 'conversion_rate': 'Conversion Rate'},
    text="conversion_rate"
)
fig.update_layout(yaxis=dict(range=[0, 1]))
fig.show()

Speaker conversion rate by language: 
shape: (4, 4)
┌────────────────────┬─────────────────┬─────────────┬─────────────────┐
│ language_displayed ┆ users_converted ┆ users_total ┆ conversion_rate │
│ ---                ┆ ---             ┆ ---         ┆ ---             │
│ str                ┆ u32             ┆ u32         ┆ f64             │
╞════════════════════╪═════════════════╪═════════════╪═════════════════╡
│ Spanish            ┆ 24              ┆ 120         ┆ 0.2             │
│ Arabic             ┆ 12              ┆ 24          ┆ 0.5             │
│ English            ┆ 926             ┆ 7169        ┆ 0.129167        │
│ German             ┆ 53              ┆ 74          ┆ 0.716216        │
└────────────────────┴─────────────────┴─────────────┴─────────────────┘


In [6]:
daily_conversion_rate = conversion_rate(marketing,'date_served')
print("Daily Conversion Rate: ", daily_conversion_rate, sep="\n")

fig = px.line(
    daily_conversion_rate.sort('date_served'), 
    x='date_served', 
    y='conversion_rate',
    title='Daily Conversion Rate',
    template='plotly_white',
    labels={'date_served': 'Date Served', 'conversion_rate': 'Conversion Rate'},
    markers=True,
)
fig.update_traces({'line_color':'green'})
fig.update_layout(yaxis=dict(range=[0, 1]))    
fig.show()

Daily Conversion Rate: 
shape: (31, 4)
┌─────────────┬─────────────────┬─────────────┬─────────────────┐
│ date_served ┆ users_converted ┆ users_total ┆ conversion_rate │
│ ---         ┆ ---             ┆ ---         ┆ ---             │
│ date        ┆ u32             ┆ u32         ┆ f64             │
╞═════════════╪═════════════════╪═════════════╪═════════════════╡
│ 0018-01-25  ┆ 23              ┆ 184         ┆ 0.125           │
│ 0018-01-22  ┆ 22              ┆ 178         ┆ 0.123596        │
│ 0018-01-16  ┆ 99              ┆ 388         ┆ 0.255155        │
│ 0018-01-19  ┆ 18              ┆ 305         ┆ 0.059016        │
│ 0018-01-28  ┆ 20              ┆ 320         ┆ 0.0625          │
│ …           ┆ …               ┆ …           ┆ …               │
│ 0018-01-07  ┆ 39              ┆ 275         ┆ 0.141818        │
│ 0018-01-13  ┆ 26              ┆ 306         ┆ 0.084967        │
│ 0018-01-10  ┆ 40              ┆ 337         ┆ 0.118694        │
│ 0018-01-01  ┆ 36              ┆ 362

In [7]:
channel_age = marketing.group_by(['marketing_channel', 'age_group']).agg(pl.col('user_id').n_unique().alias("users_num"))
print(channel_age.head())

fig = px.bar(
    channel_age.sort(['marketing_channel','age_group']), 
    x="marketing_channel", 
    y="users_num", 
    color="age_group", 
    barmode="group",  
    title="Marketing Channels by Age Group",
    labels={"marketing_channel": "Marketing Channel", "users_num": "Number of Users", 'age_group':"Age Group"},
    text="users_num"
)
fig.show()

shape: (5, 3)
┌───────────────────┬─────────────┬───────────┐
│ marketing_channel ┆ age_group   ┆ users_num │
│ ---               ┆ ---         ┆ ---       │
│ str               ┆ str         ┆ u32       │
╞═══════════════════╪═════════════╪═══════════╡
│ House Ads         ┆ 45-55 years ┆ 567       │
│ Instagram         ┆ 45-55 years ┆ 218       │
│ Push              ┆ 36-45 years ┆ 192       │
│ null              ┆ 55+ years   ┆ 2         │
│ Email             ┆ 55+ years   ┆ 53        │
└───────────────────┴─────────────┴───────────┘


In [8]:
sub_total = marketing.group_by(['date_subscribed', 'subscribing_channel']).agg(pl.col('user_id').n_unique().alias('sub_num'))
retention_subs = marketing.filter(pl.col('is_retained')==True).group_by(['date_subscribed','subscribing_channel']).agg(pl.col('user_id').n_unique().alias("users_retained"))
retention_df = retention_subs.join(sub_total,on=['date_subscribed', 'subscribing_channel'], how='inner')
retention_df = retention_df.with_columns((pl.col("users_retained")/pl.col("sub_num")).alias("retention_rate"))
retention_df = retention_df.pivot('subscribing_channel', index='date_subscribed', values='retention_rate')
retention_df = retention_df.fill_nan(0).fill_null(0)
columns = sorted(retention_df.columns)
columns.remove('date_subscribed')
print(retention_df)

for column in columns:
    fig = px.line(
        retention_df.sort('date_subscribed'), 
        x='date_subscribed', 
        y=column, 
        title=f'Daily {column} Retention Rate',
        template='plotly_white',
        labels={'date_subscribed': 'Date Subscribed', 'retention_rate': 'Retention Rate'}, 
        markers=True
    )
    fig.update_layout(yaxis=dict(range=[0, 1]))
    fig.show()

shape: (31, 6)
┌─────────────────┬───────────┬──────────┬──────────┬──────────┬───────────┐
│ date_subscribed ┆ House Ads ┆ Email    ┆ Facebook ┆ Push     ┆ Instagram │
│ ---             ┆ ---       ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ date            ┆ f64       ┆ f64      ┆ f64      ┆ f64      ┆ f64       │
╞═════════════════╪═══════════╪══════════╪══════════╪══════════╪═══════════╡
│ 0018-01-02      ┆ 0.588235  ┆ 0.75     ┆ 1.0      ┆ 1.0      ┆ 0.625     │
│ 0018-01-25      ┆ 0.833333  ┆ 0.5      ┆ 0.8      ┆ 0.0      ┆ 0.375     │
│ 0018-01-11      ┆ 0.166667  ┆ 0.333333 ┆ 0.571429 ┆ 1.0      ┆ 1.0       │
│ 0018-01-18      ┆ 0.5       ┆ 1.0      ┆ 0.666667 ┆ 0.0      ┆ 0.9       │
│ 0018-01-12      ┆ 0.0       ┆ 0.0      ┆ 0.571429 ┆ 0.333333 ┆ 0.5       │
│ …               ┆ …         ┆ …        ┆ …        ┆ …        ┆ …         │
│ 0018-01-04      ┆ 0.466667  ┆ 0.5      ┆ 0.666667 ┆ 0.0      ┆ 0.5       │
│ 0018-01-13      ┆ 0.166667  ┆ 0.333333 ┆ 0.666667 ┆ 0.5    

In [9]:
def plotting_conv(dataframe):
    columns = sorted(dataframe.columns)
    columns.remove('date_served')
    for column in columns:
        fig = px.line(
            dataframe, 
            x=dataframe.get_column('date_served'), 
            y=column, 
            title=f'Daily {column} Conversion Rate',
            labels={'date_served': 'Date served', column: 'Conversion Rate'},
            markers=True
        )
        fig.update_layout(template='plotly_white', hovermode='x', yaxis=dict(range=[0, None]))
        fig.update_traces(line_color='green')
        fig.show()

In [10]:
marketing_channel_conv = conversion_rate(marketing,['date_served','marketing_channel'])
marketing_channel_df = marketing_channel_conv.pivot('marketing_channel', index='date_served', values='conversion_rate').fill_null(0)
print(marketing_channel_df)

plotting_conv(marketing_channel_df.sort('date_served'))

shape: (31, 6)
┌─────────────┬───────────┬──────────┬──────────┬──────────┬───────────┐
│ date_served ┆ House Ads ┆ Email    ┆ Push     ┆ Facebook ┆ Instagram │
│ ---         ┆ ---       ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ date        ┆ f64       ┆ f64      ┆ f64      ┆ f64      ┆ f64       │
╞═════════════╪═══════════╪══════════╪══════════╪══════════╪═══════════╡
│ 0018-01-03  ┆ 0.088542  ┆ 0.0      ┆ 0.083333 ┆ 0.080645 ┆ 0.171875  │
│ 0018-01-15  ┆ 0.029268  ┆ 0.154791 ┆ 0.055556 ┆ 0.119403 ┆ 0.125     │
│ 0018-01-08  ┆ 0.103896  ┆ 1.0      ┆ 0.064516 ┆ 0.136364 ┆ 0.125     │
│ 0018-01-27  ┆ 0.036145  ┆ 1.0      ┆ 0.088235 ┆ 0.090909 ┆ 0.081967  │
│ 0018-01-21  ┆ 0.044248  ┆ 0.5      ┆ 0.1      ┆ 0.142857 ┆ 0.104167  │
│ …           ┆ …         ┆ …        ┆ …        ┆ …        ┆ …         │
│ 0018-01-01  ┆ 0.084656  ┆ 1.0      ┆ 0.083333 ┆ 0.117647 ┆ 0.106667  │
│ 0018-01-05  ┆ 0.126582  ┆ 1.0      ┆ 0.027778 ┆ 0.112903 ┆ 0.15942   │
│ 0018-01-16  ┆ 0.03871   ┆ 0.90566 

In [11]:
age_group_conv = conversion_rate(marketing,['date_served','age_group'])
age_group_df = age_group_conv.pivot('age_group', index='date_served', values='conversion_rate').fill_null(0)
print(age_group_df)
plotting_conv(age_group_df.sort('date_served'))

shape: (31, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ date_serve ┆ 19-24      ┆ 24-30      ┆ 30-36     ┆ 55+ years ┆ 36-45     ┆ 45-55     ┆ 0-18      │
│ d          ┆ years      ┆ years      ┆ years     ┆ ---       ┆ years     ┆ years     ┆ years     │
│ ---        ┆ ---        ┆ ---        ┆ ---       ┆ f64       ┆ ---       ┆ ---       ┆ ---       │
│ date       ┆ f64        ┆ f64        ┆ f64       ┆           ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0018-01-31 ┆ 0.105263   ┆ 0.057692   ┆ 0.0       ┆ 0.026316  ┆ 0.035088  ┆ 0.023256  ┆ 0.102041  │
│ 0018-01-24 ┆ 0.178571   ┆ 0.16129    ┆ 0.0       ┆ 0.055556  ┆ 0.09375   ┆ 0.0       ┆ 0.228571  │
│ 0018-01-23 ┆ 0.103448   ┆ 0.21875    ┆ 0.125     ┆ 0.2       ┆ 0.052632  ┆ 0.038462  ┆ 0.090909  │
│ 0018-01-27 ┆ 0.069767   ┆ 0.086957   ┆ 0.065217  ┆ 0.026316  ┆ 0.025641  ┆

In [12]:
marketing = marketing.with_columns((pl.col("date_served").dt.weekday()).alias("DoW_served"))
DoW_conversion = conversion_rate(marketing, ['DoW_served','marketing_channel'] )
DoW_conversion_df = DoW_conversion.pivot('marketing_channel', index='DoW_served', values='conversion_rate').fill_null(0)
DoW_conversion_df = DoW_conversion_df.sort('DoW_served')
print(DoW_conversion_df)

fig = px.line(
        DoW_conversion_df, 
        x='DoW_served', 
        y= DoW_conversion_df.columns, 
        title='Conversion Rate by Day of the Week',
        labels={'variable': 'Marketing Channel', 'DoW_served':'Day served','value':'Conversion Rate'},
        markers=True
)
fig.update_layout(
        xaxis=dict(tickangle=45), 
        yaxis=dict(range=[0, 1]),  
        template='plotly_white',
        
)
fig.update_xaxes(
    tickvals=[1,2,3,4,5,6,7],
    ticktext= ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
)
fig.show()

shape: (7, 6)
┌────────────┬───────────┬──────────┬──────────┬───────────┬──────────┐
│ DoW_served ┆ House Ads ┆ Email    ┆ Facebook ┆ Instagram ┆ Push     │
│ ---        ┆ ---       ┆ ---      ┆ ---      ┆ ---       ┆ ---      │
│ i8         ┆ f64       ┆ f64      ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═══════════╪══════════╪══════════╪═══════════╪══════════╡
│ 1          ┆ 0.06266   ┆ 0.162621 ┆ 0.119601 ┆ 0.122517  ┆ 0.064516 │
│ 2          ┆ 0.0703125 ┆ 0.90625  ┆ 0.147887 ┆ 0.151943  ┆ 0.115854 │
│ 3          ┆ 0.075269  ┆ 0.837209 ┆ 0.127036 ┆ 0.15016   ┆ 0.105882 │
│ 4          ┆ 0.059034  ┆ 0.727273 ┆ 0.133333 ┆ 0.143498  ┆ 0.067797 │
│ 5          ┆ 0.062278  ┆ 0.666667 ┆ 0.110132 ┆ 0.12987   ┆ 0.055556 │
│ 6          ┆ 0.057566  ┆ 0.818182 ┆ 0.109375 ┆ 0.08871   ┆ 0.069767 │
│ 7          ┆ 0.065217  ┆ 0.75     ┆ 0.116071 ┆ 0.127193  ┆ 0.065574 │
└────────────┴───────────┴──────────┴──────────┴───────────┴──────────┘


In [13]:
house_ads = marketing.filter(pl.col('marketing_channel')=='House Ads')
conv_lang_channel = conversion_rate(house_ads,['date_served','language_displayed'])
conv_lang_df = conv_lang_channel.pivot('language_displayed', index='date_served', values='conversion_rate').fill_null(0)
print(conv_lang_df)
plotting_conv(conv_lang_df.sort('date_served'))

shape: (31, 5)
┌─────────────┬──────────┬────────┬──────────┬────────┐
│ date_served ┆ English  ┆ Arabic ┆ Spanish  ┆ German │
│ ---         ┆ ---      ┆ ---    ┆ ---      ┆ ---    │
│ date        ┆ f64      ┆ f64    ┆ f64      ┆ f64    │
╞═════════════╪══════════╪════════╪══════════╪════════╡
│ 0018-01-22  ┆ 0.070588 ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ 0018-01-19  ┆ 0.033784 ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ 0018-01-01  ┆ 0.075145 ┆ 1.0    ┆ 0.0      ┆ 0.2    │
│ 0018-01-05  ┆ 0.118881 ┆ 0.0    ┆ 0.214286 ┆ 0.0    │
│ 0018-01-16  ┆ 0.03871  ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ …           ┆ …        ┆ …      ┆ …        ┆ …      │
│ 0018-01-21  ┆ 0.044248 ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ 0018-01-31  ┆ 0.039216 ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ 0018-01-12  ┆ 0.027211 ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ 0018-01-27  ┆ 0.036364 ┆ 0.0    ┆ 0.0      ┆ 0.0    │
│ 0018-01-09  ┆ 0.129252 ┆ 0.0    ┆ 0.1      ┆ 0.0    │
└─────────────┴──────────┴────────┴──────────┴────────┘


In [14]:
#house_ads = house_ads.with_columns(pl.when(pl.col("language_displayed") == pl.col("language_preferred")).then("Yes").otherwise("No").alias("is_correct_lang"))

#language_check = house_ads.groupby(['date_served','is_correct_lang'])['is_correct_lang'].count()

#print(language_check_df)

# Divide the count where language is correct by the row sum
#language_check_df['pct'] = language_check_df['Yes'] / language_check_df.sum(axis=1)

#fig = px.line(
#        language_check_df, 
#        x=language_check_df.index, 
#        y=language_check_df['pct'], 
#        title='Percentage of users being served ads in the right language',
#        template='plotly_white',
#        labels={'pct': 'Percentage', 'date_served':'Date served'},
#        markers=True
#)
#fig.update_traces(line_color='green')
#fig.update_layout(
#        xaxis=dict(tickangle=45),  
#        yaxis=dict(range=[0, None]),  
#)
#fig.show()